<h3>Forex Prediction: Recursive Neural Networks (RNNs)

In [4]:
import pandas as pd
import numpy as np
from data_functions import *

In [5]:
all_data, cc_dict, countries =load_and_process_forex_data()
all_data.head()

,Date,EURO/US$,UNITED KINGDOM POUND/US$,YEN/US$,YUAN/US$,AUSTRALIAN DOLLAR/US$
0,2000-01-03,0.9847,0.6146,101.70,8.2798,1.5172
1,2000-01-04,0.9700,0.6109,103.09,8.2799,1.5239
2,2000-01-05,0.9676,0.6092,103.77,8.2798,1.5267
3,2000-01-06,0.9686,0.6070,105.19,8.2797,1.5291
4,2000-01-07,0.9714,0.6104,105.17,8.2794,1.5272


In [6]:
def create_data_dict_currency(data,countries,currency_dict):

    def extract_currency_data(data, currency):
            """
            Extract a specific country's data from the main dataframe.
            
            Parameters:
            data (DataFrame): The main dataframe containing all countries' data
            country_name (str): The name of the country to extract data for
            
            Returns:
            DataFrame: A dataframe containing only the specified country's data with date column
            """
            country_data = data[['Date',currency]].copy()
            
            # Ensure date column is included and properly formatted
            
            return country_data

    data_dict = {}
    for country in countries:
        data_dict[currency_dict[country]] = extract_currency_data(data,currency_dict[country])
    
    return data_dict    

In [7]:
# Create a dictionary of dataframes of each currency
data_dict = create_data_dict_currency(all_data,countries,cc_dict)
data_dict['EURO/US$']

,Date,EURO/US$
0,2000-01-03,0.9847
1,2000-01-04,0.9700
2,2000-01-05,0.9676
3,2000-01-06,0.9686
4,2000-01-07,0.9714
...,...,...
5211,2019-12-24,0.9022
5213,2019-12-26,0.9007
5214,2019-12-27,0.8949
5215,2019-12-30,0.8915


<h5>Pre-processing to carry out:</h5>


- Train-test split.
- Scaling (will use a RobustScaler()). Fit_transform on train, and transform on test.
- Time window creation.


N.B. Save the scalers used for each currency to be used for data preparation and inversion when using the model in the app (as pickle files).

In [8]:
# Train-test splitting (non-random, last 60 rows are taken as the test). Will focus on EUR now:
df = data_dict['EURO/US$'].copy()
if 'Date' in df.columns:
    df = df.sort_values('Date').reset_index(drop=True)

test_df = df.tail(60).reset_index(drop=True)
train_df = df.iloc[:-60].reset_index(drop=True)

In [9]:
from sklearn.preprocessing import RobustScaler

def robust_scale_train_test(train_df, test_df, drop_date=True):
    """
    Fit RobustScaler on train_df (drop Date column if present) and transform both train and test.
    Returns: scaler, train_scaled_df, test_scaled_df
    """
    if drop_date and 'Date' in train_df.columns:
        X_train = train_df.drop(columns=['Date']).copy()
    else:
        X_train = train_df.copy()
    if drop_date and 'Date' in test_df.columns:
        X_test = test_df.drop(columns=['Date']).copy()
    else:
        X_test = test_df.copy()

    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train.values)
    X_test_scaled = scaler.transform(X_test.values)

    train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

    return scaler, train_scaled_df, test_scaled_df

In [10]:
eur_scaler, train_scaled_df, test_scaled_df = robust_scale_train_test(train_df,test_df)

In [11]:
# Create windows for an LSTM model:
def LSTM_input(df, input_sequence):
    """
    Generate supervised learning sequences from a time-ordered dataset
    for one-step-ahead LSTM forecasting.

    This function converts a time-series DataFrame into sliding input
    sequences (X) and corresponding target values (y), suitable for
    training a many-to-one LSTM model.

    For each sample:
        - X contains `input_sequence` consecutive past observations
        - y contains the immediately following observation

    The function assumes the data is:
        - Ordered in ascending chronological order (oldest → newest)
        - Free of non-numeric columns (e.g. time columns removed)
        - Already scaled or normalised, if required

    Parameters
    ----------
    df : pandas.DataFrame
        Time-series data containing one or more numeric features.
        Shape: (n_samples, n_features).
        The index or original time column is not used by this function.

    input_sequence : int
        Number of past time steps to include in each input sequence
        (i.e. the LSTM lookback window).

    Returns
    -------
    X : numpy.ndarray
        Array of input sequences with shape:
            (n_samples - input_sequence, input_sequence, n_features)

    y : numpy.ndarray
        Array of target values with shape:
            (n_samples - input_sequence, n_features)

    Notes
    -----
    - This function performs one-step-ahead forecasting.
    - Each target value corresponds to the time step immediately
      following its input sequence.
    - The function does not shuffle data and preserves temporal order.
    - The function does not perform any scaling or missing-value handling.

    Example
    -------
    >>> X, y = Sequential_Input_LSTM(df_scaled, input_sequence=28)
    >>> X.shape
    (num_samples, 28, num_features)
    >>> y.shape
    (num_samples, num_features)
    """
    df_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df_np) - input_sequence):
        row = [a for a in df_np[i:i + input_sequence]]
        X.append(row)
        label = df_np[i + input_sequence]
        y.append(label)
        
    return np.array(X), np.array(y)

In [12]:
# Generate the windowed train and test (scaled) data:
train_windowed_X, train_windowed_y = LSTM_input(train_scaled_df,10)
test_windowed_X, test_windowed_y = LSTM_input(test_scaled_df, 10)

In [13]:
# Data is now split and preprocessed ready for modelling.

<h5>Modelling

- Will create an optimise a model for the EUR/USD set and then train the same type of model on the other currencies.

- Will use a temporal split for the validation instead of just creating a validation set (from the training set, manually) and providing it i.e. I will define a fraction (of the last rows) that will be taken by the model for validation from the training set. Validation isn't a necessity but it is useful for model evaluation.

- Will start with a single LSTM layer and then add more layers if evaluation shows that it is necessary.

In [14]:
import tensorflow as tf
# Check if the GPU is being used:
print("TF version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))
# Import others:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

TF version: 2.20.0
GPUs: []


In [17]:
# May need to do training on the PC (not mac) as the training speed is poor on mac. Cunrrently running
# python 3.12 which isn't compatiable with tensorflow-metal (allows GPU training on the Mac M3 chip).

In [ ]:
# Starting with a single LSTM layer model
X_train = train_windowed_X
y_train = train_windowed_y

lookback_length = X_train.shape[1] # Should be 10 here.
features = X_train.shape[2] # Should be 1 here.

print(lookback_length)
print(features)

Train_1 = False
while Train_1 == True:

    eur_model_1 = keras.Sequential([
        keras.layers.Input(shape=(lookback_length,features)),
        keras.layers.LSTM(32, return_sequences=False), # If not False, the dimnesionality of the predictions can be = rows*look_back_length
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation = 'linear') #This is a regression problem of continuous and unbound values. No artificial constraints should be placed.
    ])

    eur_model_1.summary()

    # Train the model:
    # Model training here is slow. Make sure to optimise the model structure and compilation/training parameters.
    # Early stopping to reduce overfitting:

    early_stop = EarlyStopping(monitor = 'val_loss', patience = 5)

    eur_model_1.compile(loss = MeanSquaredError(),
                        optimizer = Adam(learning_rate = 0.01), 
                        metrics = [RootMeanSquaredError()])

    eur_model_1_history = eur_model_1.fit(
        X_train, y_train,
        validation_split=0.1, # Last 10% of the training set is taken as the validation set: a 'temporal' split.
        epochs=50,
        batch_size=256,
        shuffle=False,
        callbacks=[early_stop],
        verbose=1
        )

10
1


Forecasting Functionality:

Things needed for forecasting in the streamlit app:
- A way to load the model from a pkl file.
- For 'execute_rnn()', a way to use the model, processed data (i.e. windowed) for a single currency, and a defined forecast length.
- The processed data (windowed) should be generated when the app.py is first run. A function to generate windowed data for all currencies should be run in the 'if __name__ == etc.. ' block. Call it something similar to this:

    xgbdata_raw = create_data_dict_currency_xgboost(data,country_names,country_currency_dict)
    xgbdata_processed = process_all_xgboost(xgbdata_raw)

In [18]:
def incremental_forecast_to_df(
    model,
    history,
    lookback,
    horizon=60,
    scaler=None,
    start_step=1,
    column_name="forecast",
):
    """
    Incremental (recursive) forecasting for a univariate windowed TF RNN model.

    Assumptions:
      - Univariate series (single column)
      - Model input shape: (batch, lookback, 1)
      - Model output: next value as (batch, 1) or (batch,)

    Parameters
    ----------
    model:
        A pre-trained tf.keras model (or compatible object with .predict()).
    history:
        Most recent observed values. If scaler is provided, these should be UNscaled.
        Must contain at least `lookback` points.
    lookback:
        Window length used during training.
    horizon:
        Forecast length, e.g. 60 "days" (steps).
    scaler:
        Optional fitted scaler (e.g. sklearn StandardScaler) fit on training data.
        If provided, history is scaled internally; outputs are inverse-transformed.
    start_step:
        Starting integer index for the forecast steps (default 1).
    column_name:
        Name of the forecast column in the output DataFrame.

    Returns
    -------
    pd.DataFrame with integer index [start_step ... start_step + horizon - 1]
    """
    # --- validate & coerce history ---
    hist = np.asarray(history, dtype=np.float32).reshape(-1, 1)
    if hist.shape[0] < lookback:
        raise ValueError(
            f"`history` must have at least {lookback} observations; got {hist.shape[0]}."
        )
    if horizon < 1:
        raise ValueError("`horizon` must be >= 1.")
    if lookback < 1:
        raise ValueError("`lookback` must be >= 1.")

    # --- scale if needed ---
    if scaler is not None:
        hist_scaled = scaler.transform(hist)  # shape (n, 1)
    else:
        hist_scaled = hist  # already in training scale

    # --- rolling window (scaled) ---
    window = hist_scaled[-lookback:].astype(np.float32).reshape(1, lookback, 1)

    preds_scaled = np.zeros((horizon, 1), dtype=np.float32)

    for t in range(horizon):
        yhat = model.predict(window, verbose=0)
        yhat = np.asarray(yhat).reshape(-1)  # handles (1,1) or (1,)
        yhat_val = np.float32(yhat[0])

        preds_scaled[t, 0] = yhat_val

        # update window: drop oldest timestep, append prediction
        window = np.concatenate([window[:, 1:, :], [[[yhat_val]]]], axis=1)

    # --- inverse transform if scaler provided ---
    if scaler is not None:
        preds = scaler.inverse_transform(preds_scaled)
    else:
        preds = preds_scaled

    # --- build DataFrame with integer step index ---
    steps = np.arange(start_step, start_step + horizon, dtype=int)
    out = pd.DataFrame({column_name: preds.reshape(-1)}, index=steps)
    out.index.name = "step"
    return out


def forecast_from_pickled_model_to_df(
    model,
    history,
    lookback,
    horizon=60,
    scaler=None,
    start_step=1,
    column_name="forecast",
):
    """
    Convenience wrapper:
      - Provide a pre-trained model
      - Runs incremental forecast
      - Returns results as a DataFrame with integer step index
    """
    return incremental_forecast_to_df(
        model=model,
        history=history,
        lookback=lookback,
        horizon=horizon,
        scaler=scaler,
        start_step=start_step,
        column_name=column_name,
    )